# <b style="color: white; background-color: #00bbff; padding: 5px 10px; border-radius: 5px;">LIBRARY</b>

In [2]:
import os
from dotenv import load_dotenv
from pyspark.sql import SparkSession

# Carrega as variáveis de ambiente
load_dotenv()
s3_endpoint = os.getenv("S3_ENDPOINT")
s3_access_key = os.getenv("S3_ACCESS_KEY")
s3_secret_key = os.getenv("S3_SECRET_KEY")

print(s3_access_key)

# Inicializa SparkSession com suporte a Iceberg + MinIO (igual ao seu script original)
spark = SparkSession.builder \
    .appName("ReadIcebergSilverTables") \
    .config("spark.jars", "/opt/spark/jars/iceberg-spark-runtime-3.5_2.12-1.6.0.jar") \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .config("spark.sql.catalog.local", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.local.type", "hadoop") \
    .config("spark.sql.catalog.local.warehouse", "s3a://datalake/iceberg") \
    .config("spark.hadoop.fs.s3a.endpoint", s3_endpoint) \
    .config("spark.hadoop.fs.s3a.access.key", s3_access_key) \
    .config("spark.hadoop.fs.s3a.secret.key", s3_secret_key) \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

minioadmin


# <b style="color: white; background-color: #00bbff; padding: 5px 10px; border-radius: 5px;">READING TABLES BRONZE</b>

In [3]:
# Lista tabelas da camada silver
bronze_tables = [row.tableName for row in spark.sql("SHOW TABLES IN local.bronze").collect()]

print("\n📊 Tabelas disponíveis na camada Bronze:")
for table in bronze_tables:
    print(f"- {table}")

# Exemplo: Ler uma tabela específica (você pode escolher qualquer uma da lista)
if bronze_tables:
    table_to_read = bronze_tables[0]  # Lendo a primeira tabela como exemplo
    print(f"\n🔍 Lendo dados da tabela: local.bronze.{table_to_read}")
    
    df = spark.read.format("iceberg").load(f"local.bronze.{table_to_read}")
    
    # Mostra o schema e algumas linhas
    print("\n📐 Schema:")
    df.printSchema()
    
    print("\n📝 Amostra dos dados (10 primeiras linhas):")
    df.show(10, truncate=False)
    
    # Você também pode fazer consultas SQL diretamente
    print("\n🔎 Exemplo de consulta SQL (contagem de registros):")
    spark.sql(f"SELECT COUNT(*) FROM local.bronze.{table_to_read}").show()
else:
    print("\n⚠️ Nenhuma tabela encontrada na camada Bronze.")

print("\n✅ Leitura concluída.")


📊 Tabelas disponíveis na camada Bronze:
- clientes
- vendas

🔍 Lendo dados da tabela: local.bronze.clientes

📐 Schema:
root
 |-- id: string (nullable = true)
 |-- nome: string (nullable = true)
 |-- email: string (nullable = true)
 |-- data_cadastro: string (nullable = true)
 |-- status: string (nullable = true)
 |-- created_at: timestamp (nullable = true)


📝 Amostra dos dados (10 primeiras linhas):


+-------+--------------------------+----------------------------------------+-------------+------+--------------------------+
|id     |nome                      |email                                   |data_cadastro|status|created_at                |
+-------+--------------------------+----------------------------------------+-------------+------+--------------------------+
|6010006|Anthony Gabriel Cavalcante|anthony.gabriel.cavalcante.2@exemplo.com|2025-03-22   |ativo |2025-09-13 19:02:41.891835|
|6010009|José Albuquerque          |josé.albuquerque.5@exemplo.com          |2025-03-09   |ativo |2025-09-13 19:02:41.891835|
|6010017|Hellena Alves             |hellena.alves.13@exemplo.com            |2025-02-03   |ativo |2025-09-13 19:02:41.891835|
|6010021|Letícia Nunes             |letícia.nunes.17@exemplo.com            |2025-01-15   |ativo |2025-09-13 19:02:41.891835|
|6010024|Maria Flor Brito          |maria.flor.brito.20@exemplo.com         |2025-01-21   |ativo |2025-09-13 19:02:41.

+--------+
|count(1)|
+--------+
|   14742|
+--------+


✅ Leitura concluída.


# <b style="color: white; background-color: #00bbff; padding: 5px 10px; border-radius: 5px;">READING TABLES SILVER</b>

In [5]:
# Lista tabelas da camada silver
bronze_tables = [row.tableName for row in spark.sql("SHOW TABLES IN local.bronze").collect()]

print("\n📊 Tabelas disponíveis na camada Bronze:")
for table in bronze_tables:
    print(f"- {table}")

# Exemplo: Ler uma tabela específica (você pode escolher qualquer uma da lista)
if bronze_tables:
    table_to_read = bronze_tables[0]  # Lendo a primeira tabela como exemplo
    print(f"\n🔍 Lendo dados da tabela: local.bronze.{table_to_read}")
    
    df = spark.read.format("iceberg").load(f"local.bronze.{table_to_read}")
    
    # Mostra o schema e algumas linhas
    print("\n📐 Schema:")
    df.printSchema()
    
    print("\n📝 Amostra dos dados (10 primeiras linhas):")
    df.show(10, truncate=False)
    
    # Você também pode fazer consultas SQL diretamente
    print("\n🔎 Exemplo de consulta SQL (contagem de registros):")
    spark.sql(f"SELECT COUNT(*) FROM local.bronze.{table_to_read}").show()
else:
    print("\n⚠️ Nenhuma tabela encontrada na camada Bronze.")

print("\n✅ Leitura concluída.")


📊 Tabelas disponíveis na camada Bronze:
- clientes
- vendas

🔍 Lendo dados da tabela: local.bronze.clientes

📐 Schema:
root
 |-- id: string (nullable = true)
 |-- nome: string (nullable = true)
 |-- email: string (nullable = true)
 |-- data_cadastro: string (nullable = true)
 |-- status: string (nullable = true)
 |-- created_at: timestamp (nullable = true)


📝 Amostra dos dados (10 primeiras linhas):


+-------+--------------------------+----------------------------------------+-------------+------+--------------------------+
|id     |nome                      |email                                   |data_cadastro|status|created_at                |
+-------+--------------------------+----------------------------------------+-------------+------+--------------------------+
|6010006|Anthony Gabriel Cavalcante|anthony.gabriel.cavalcante.2@exemplo.com|2025-03-22   |ativo |2025-09-13 19:02:41.891835|
|6010009|José Albuquerque          |josé.albuquerque.5@exemplo.com          |2025-03-09   |ativo |2025-09-13 19:02:41.891835|
|6010017|Hellena Alves             |hellena.alves.13@exemplo.com            |2025-02-03   |ativo |2025-09-13 19:02:41.891835|
|6010021|Letícia Nunes             |letícia.nunes.17@exemplo.com            |2025-01-15   |ativo |2025-09-13 19:02:41.891835|
|6010024|Maria Flor Brito          |maria.flor.brito.20@exemplo.com         |2025-01-21   |ativo |2025-09-13 19:02:41.

+--------+
|count(1)|
+--------+
|   14742|
+--------+


✅ Leitura concluída.


# <b style="color: white; background-color: #00bbff; padding: 5px 10px; border-radius: 5px;">READING TABLES GOLD</b>

In [7]:
# Lista tabelas da camada silver
gold_tables = [row.tableName for row in spark.sql("SHOW TABLES IN local.gold").collect()]

print("\n📊 Tabelas disponíveis na camada Gold:")
for table in gold_tables:
    print(f"- {table}")

# Exemplo: Ler uma tabela específica (você pode escolher qualquer uma da lista)
if gold_tables:
    table_to_read = gold_tables[0]  # Lendo a primeira tabela como exemplo
    print(f"\n🔍 Lendo dados da tabela: local.gold.{table_to_read}")
    
    df = spark.read.format("iceberg").load(f"local.gold.{table_to_read}")
    
    # Mostra o schema e algumas linhas
    print("\n📐 Schema:")
    df.printSchema()
    
    print("\n📝 Amostra dos dados (10 primeiras linhas):")
    df.show(10, truncate=False)
    
    # Você também pode fazer consultas SQL diretamente
    print("\n🔎 Exemplo de consulta SQL (contagem de registros):")
    spark.sql(f"SELECT COUNT(*) FROM local.gold.{table_to_read}").show()
else:
    print("\n⚠️ Nenhuma tabela encontrada na camada Gold.")

print("\n✅ Leitura concluída.")


📊 Tabelas disponíveis na camada Gold:
- dim_clientes
- fato_vendas
- agregado_vendas_categoria
- agregado_vendas_cliente

🔍 Lendo dados da tabela: local.gold.dim_clientes

📐 Schema:
root
 |-- cliente_id: string (nullable = true)
 |-- nome_cliente: string (nullable = true)
 |-- email: string (nullable = true)
 |-- data_cadastro: string (nullable = true)
 |-- ativo: integer (nullable = true)
 |-- ano_cadastro: integer (nullable = true)
 |-- mes_cadastro: integer (nullable = true)
 |-- created_at: timestamp (nullable = true)


📝 Amostra dos dados (10 primeiras linhas):


+----------+-----------------------+--------------------------------------+-------------+-----+------------+------------+--------------------------+
|cliente_id|nome_cliente           |email                                 |data_cadastro|ativo|ano_cadastro|mes_cadastro|created_at                |
+----------+-----------------------+--------------------------------------+-------------+-----+------------+------------+--------------------------+
|6010021   |Letícia Nunes          |letícia.nunes.17@exemplo.com          |2025-01-15   |1    |2025        |1           |2025-09-13 19:02:41.891835|
|6010024   |Maria Flor Brito       |maria.flor.brito.20@exemplo.com       |2025-01-21   |1    |2025        |1           |2025-09-13 19:02:41.891835|
|6010031   |João Vitor da Cruz     |joão.vitor.da.cruz.27@exemplo.com     |2025-01-18   |1    |2025        |1           |2025-09-13 19:02:41.891835|
|6010040   |Joana da Rosa          |joana.da.rosa.36@exemplo.com          |2025-01-14   |1    |2025       

# <b style="color: white; background-color: #00bbff; padding: 5px 10px; border-radius: 5px;">READING TABLES GOLD AGREGADOS</b>

In [8]:
# Lista tabelas da camada silver
gold_tables = [row.tableName for row in spark.sql("SHOW TABLES IN local.gold").collect()]

print("\n📊 Tabelas disponíveis na camada Gold:")
for table in gold_tables:
    print(f"- {table}")

# Exemplo: Ler uma tabela específica (você pode escolher qualquer uma da lista)
if gold_tables:
    table_to_read = gold_tables[2]  # Lendo a primeira tabela como exemplo
    print(f"\n🔍 Lendo dados da tabela: local.gold.{table_to_read}")
    
    df = spark.read.format("iceberg").load(f"local.gold.{table_to_read}")
    
    # Mostra o schema e algumas linhas
    print("\n📐 Schema:")
    df.printSchema()
    
    print("\n📝 Amostra dos dados (10 primeiras linhas):")
    df.show(10, truncate=False)
    
    # Você também pode fazer consultas SQL diretamente
    print("\n🔎 Exemplo de consulta SQL (contagem de registros):")
    spark.sql(f"SELECT COUNT(*) FROM local.gold.{table_to_read}").show()
else:
    print("\n⚠️ Nenhuma tabela encontrada na camada Gold.")

print("\n✅ Leitura concluída.")


📊 Tabelas disponíveis na camada Gold:
- dim_clientes
- fato_vendas
- agregado_vendas_categoria
- agregado_vendas_cliente

🔍 Lendo dados da tabela: local.gold.agregado_vendas_categoria

📐 Schema:
root
 |-- categoria: string (nullable = true)
 |-- ano_venda: integer (nullable = true)
 |-- mes_venda: integer (nullable = true)
 |-- total_vendido: double (nullable = true)
 |-- quantidade_vendas: long (nullable = true)
 |-- valor_medio_venda: double (nullable = true)
 |-- created_at: timestamp (nullable = true)


📝 Amostra dos dados (10 primeiras linhas):


+-----------+---------+---------+--------------------+-----------------+------------------+-------------------------+
|categoria  |ano_venda|mes_venda|total_vendido       |quantidade_vendas|valor_medio_venda |created_at               |
+-----------+---------+---------+--------------------+-----------------+------------------+-------------------------+
|perifericos|2025     |1        |1969373.2999999968  |4590             |429.05736383442195|2025-09-13 19:36:21.76979|
|eletronicos|2025     |1        |3.7992888979999684E7|11530            |3295.1334761491485|2025-09-13 19:36:21.76979|
|mobilia    |2025     |1        |9307029.5           |4579             |2032.5462983184102|2025-09-13 19:36:21.76979|
|acessorios |2025     |1        |1040192.3499999995  |2339             |444.7166951688754 |2025-09-13 19:36:21.76979|
+-----------+---------+---------+--------------------+-----------------+------------------+-------------------------+


🔎 Exemplo de consulta SQL (contagem de registros):
+--